In [1]:
#pip install latex

In [2]:
#pip install jdc

In [3]:
#import jdc
#import os
#import 

# Run OpenFOAM

In [4]:
################# Parent Class for running OpenFOAM from python  #####################################

#### Seperate runSim class architecture 
#from os import path
import os
import subprocess

from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile
from PyFoam.Basics.DataStructures import Vector

class RunOFv4():
                            
    def __init__(self, case, x, gen, solver, mesher, procLim, nProc):#, ind=0):
        self.case = case
        # array of variables for this generation's population
        self.x = x
        # generation number
        self.gen = gen
        ##### Pre-Process/Execute/Post-Process Objects #####
        self.preProc = PreProcess(x, gen, mesher, nProc)
        self.exec = ExecuteSimulations(x, gen, solver, procLim, nProc)
        self.postProc = PostProcess(x, gen)
        
        #os.mkdir('./cases/gen%i' %self.gen)
            
    def runGen(self):
        ##### SET-UP CASE FILES #####
        #print("PRE-PROCESS")
        self.preProc.main(case=self.case)
        #### EXECUTE SIMULATIONS #####
        #print("EXECUTE")
        # decide to run cases in parallel or serial
        self.exec.run()
        ##### POST-PROCESS #####
        #print("POST-PROCESS")
        obj = self.postProc.main(case=self.case)
        
        return obj

# Pre-Process 

In [5]:
#header = OF_header('blockMeshDic')
#print(header)

In [6]:
def OF_header(obj, clas='dictionary', ver='2.0', form='ascii'):
    return """
/*--------------------------------*- C++ -*----------------------------------*
| =========                |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  5                                     |
|   \\  /    A nd           | Web:      www.OpenFOAM.org                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     %s;
    format      %s;
    class       %s;
    object      %s;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //
"""%(obj, clas, ver, form)

def OF_decomposeParDict(nProc, method='scotch'):
    return OF_header('decomposeParDict')+'''
numberOfSubdomains %i;

method          %s;

// ************************************************************************* //
'''%(nProc, method)

def OF_fvSchemes(ddtSchemes='steadyState',gradSchemes='Gauss linear'):
    return OF_header('fvSchemes')+'''
ddtSchemes
{
    default         %s;
}

gradSchemes
{
    default         %s;
}

divSchemes
{
    default         none;
    div(phi,U)      bounded Gauss linearUpwind grad(U);
    div(phi,nuTilda) bounded Gauss linearUpwind grad(nuTilda);
    div((nuEff*dev2(T(grad(U))))) Gauss linear;
}

laplacianSchemes
{
    default         Gauss linear corrected;
}

interpolationSchemes
{
    default         linear;
}

snGradSchemes
{
    default         corrected;
}

wallDist
{
    method meshWave;
}


// ************************************************************************* //
'''%(ddtSchemes, gradSchemes)

#print(OF_decomposeParDict(2))
#def OF_parameters():
#def OF_vetices():
#def OF_boundaries():    

### Airfoil blockMeshDic

In [7]:
### blockMeshDic airfoil 
# Header
af_bMD0 = """
/*--------------------------------*- C++ -*----------------------------------*
| =========                |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  5                                     |
|   \\  /    A nd           | Web:      www.OpenFOAM.org                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      blockMeshDict;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

convertToMeters 1;
"""
# Vertices
af_bMD1 = """
convertToMeters 1;

vertices
(
    ($pc	$rNeg 	$minZ)		// point 0
    (1.0 	$rNeg 	$minZ)		// point 1
    ($l 	$rNeg 	$minZ)		// point 2
    ($pc 	$ypcNeg	$minZ)		// point 3
    ($rNeg 	0.0		$minZ)		// point 4
    (0.0 	0.0		$minZ)		// point 5
    (1.0	0.0		$minZ)		// point 6
    ($l 	0.0		$minZ)		// point 7
    ($pc 	$ypcPos	$minZ)		// point 8
    ($pc 	$rPos	$minZ)		// point 9
    (1.0	$rPos	$minZ)		// point 10
    ($l 	$rPos	$minZ)		// point 11

    ($pc	$rNeg 	$maxZ)		// point 12
    (1.0 	$rNeg 	$maxZ)		// point 13
    ($l 	$rNeg 	$maxZ)		// point 14
    ($pc 	$ypcNeg	$maxZ)		// point 15
    ($rNeg 	0.0		$maxZ)		// point 16
    (0.0 	0.0		$maxZ)		// point 17
    (1.0	0.0		$maxZ)		// point 18
    ($l 	0.0		$maxZ)		// point 19
    ($pc 	$ypcPos	$maxZ)		// point 20
    ($pc 	$rPos	$maxZ)		// point 21
    (1.0	$rPos	$maxZ)		// point 22
    ($l 	$rPos	$maxZ)		// point 23
);\n """

# Blocks
af_bMD2 = """
blocks
(
    hex (17 15 12 16 5 3 0 4) 	($Npre 		$Ny 	$Nz) 	simpleGrading 		// block 0
    ( 
        (
            (0.5 0.65 $pre1)
            (0.5 0.35 $pre2)
        )
        (
            (0.2 0.5 $yDir1)
            (0.8 0.5 $yDir2)
        )
        (
            (1 1 $zDir)
        )
    )
    hex (5 8 9 4 17 20 21 16) 	($Npre 		$Ny 	$Nz) 	simpleGrading 		// block 1
    ( 
        (
            (0.5 0.65 $pre1)
            (0.5 0.35 $pre2)
        )
        (
            (0.2 0.5 $yDir1)
            (0.8 0.5 $yDir2)
        )
        (
            (1 1 $zDir)
        )
    )
    hex (15 18 13 12 3 6 1 0) 	($Npost 	$Ny 	$Nz) 	simpleGrading 		// block 2
    ( 
        (
            (0.5 0.45 $post1)
            (0.5 0.55 $post2)
        )
        (
            (0.2 0.5 $yDir1)
            (0.8 0.5 $yDir2)
        )
        (
            (1 1 $zDir)
        )
    )
    hex (8 6 10 9 20 18 22 21) 	($Npost 	$Ny 	$Nz) 	simpleGrading 		// block 3
    ( 
        (
            (0.5 0.45 $post1)
            (0.5 0.55 $post2)
        )
        (
            (0.2 0.5 $yDir1)
            (0.8 0.5 $yDir2)
        )
        (
            (1 1 $zDir)
        )
    )
    hex (18 19 14 13 6 7 2 1) 	($Nwake 	$Ny 	$Nz) 	simpleGrading 		// block 4
    ( 
        (
            (1 1 $wake)
        )
        (
            (0.2 0.5 $yDir1)
            (0.8 0.5 $yDir2)
        )
        (
            (1 1 $zDir)
        )
    )
    hex (6 7 11 10 18 19 23 22)	($Nwake 	$Ny 	$Nz) 	simpleGrading 		// block 5
    ( 
        (
            (1 1 $wake)
        )
        (
            (0.2 0.5 $yDir1)
            (0.8 0.5 $yDir2)
        )
        (
            (1 1 $zDir)
        )
    )
); \n
"""

# Edges are included below

# Boundary
af_bMD3 = """
boundary
(
    inlet
    {
        type patch;
        faces
        (
            (9 21 16 4)
            (4 16 12 0)
        );
    }   

    outlet
    {
        type patch;
        faces
        (
            (23 11 7 19)
            (19 7 2 14)
        );
    }   


    airfoil
    {
        type wall;
        faces
        (
            (8 20 18 6)
            (6 18 15 3)
            (17 5 3 15)
            (20 8 5 17)
        );
    }   

    lower
    {
        type patch;
        faces
        (
            (0 12 13 1)
            (1 13 14 2)
        );
    }   

    upper
    {
        type patch;
        faces
        (
            (21 9 10 22)
            (22 10 11 23)
        );
    }

);

// ************************************************************************* //
"""



### blockMeshDic Diffuser

In [8]:
################### blockMeshDic Structure ##################################
# blockMeshDic Header
diff_bMD1 = """
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  5                                     |
|   \\  /    A nd           | Web:      www.OpenFOAM.org                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      blockMeshDict;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //
"""

# Body
diff_bMD2 = """
z1  0.0; 
z2  0.1; 
Nup   110;
Nuu   90;
Ncowl 20;
Naxis 200;
Ninf  140;
Nramp 300;
Ndown 400;

convertToMeters 1;

vertices
(
    (1.0	0.0		$z1) // Point 0
    (2.0	0.0		$z1) // Point 1
    ($DLcO	$LsO	$z1) // Point 2
    ($TLcO	0.1		$z1) // Point 3
    (1.0	0.8		$z1) // Point 4
    (2.0	0.8		$z1) // Point 5
    ($DLcO	0.8		$z1) // Point 6
    ($TLcO	0.8		$z1) // Point 7
    (1.0	0.85	$z1) // Point 8
    (2.0	0.85	$z1) // Point 9
    ($ULcO	0.85	$z1) // Point 10
    ($TLcO	0.85	$z1) // Point 11
    (1.0	2.5   	$z1) // Point 12
    (2.0	2.5	    $z1) // Point 13
    ($ULcO	2.5	    $z1) // Point 14
    ($TLcO	2.5	    $z1) // Point 15

    (1.0	0.0		$z2) // Point 16
    (2.0	0.0		$z2) // Point 17
    ($DLcO	$LsO	$z2) // Point 18
    ($TLcO	0.1		$z2) // Point 19
    (1.0	0.8		$z2) // Point 20
    (2.0	0.8		$z2) // Point 21
    ($DLcO	0.8		$z2) // Point 22
    ($TLcO	0.8		$z2) // Point 23
    (1.0	0.85	$z2) // Point 24
    (2.0	0.85	$z2) // Point 25
    ($ULcO	0.85	$z2) // Point 26
    ($TLcO	0.85	$z2) // Point 27
    (1.0	2.5   	$z2) // Point 28
    (2.0	2.5     $z2) // Point 29
    ($ULcO	2.5	    $z2) // Point 30
    ($TLcO	2.5	    $z2) // Point 31
);

blocks
(
    //block 0
    hex (0 1 5 4 16 17 21 20) ($Ninf $Naxis 1) simpleGrading
    (
        0.27
        (
            (0.10		0.15	1)
            (0.40		0.35	3.2)
            (0.40		0.35	0.3125)
            (0.10		0.15	1)
        )   	
        1
    )    
    //block 1
    hex (1 2 6 5 17 18 22 21) ($Nramp $Naxis 1) simpleGrading
    (
        1
        (
            (0.10		0.15	1)
            (0.40		0.35	3.2)
            (0.40		0.35	0.3125)
            (0.10		0.15	1)
        )   	
        1
    )
    //block 2
    hex (2 3 7 6 18 19 23 22) ($Ndown $Naxis 1) simpleGrading 		
    (
        (
            (0.50       0.50    3.125)
            (0.50       0.50    0.32)
        )
        (
            (0.10		0.15	1)
            (0.40		0.35	3.2)
            (0.40		0.35	0.3125)
            (0.10		0.15	1)
        )   	
        1
    )
    //block 3
    hex (4 5 9 8 20 21 25 24) ($Ninf $Ncowl 1) simpleGrading (0.27 1 1)
    //block 4
    hex (5 6 10 9 21 22 26 25) ($Nramp $Ncowl 1) simpleGrading (1 1 1)
    //block 5
    hex (8 9 13 12 24 25 29 28) ($Ninf $Nup 1) simpleGrading (0.27 30 1)
    //block 6
    hex (9 10 14 13 25 26 30 29) ($Nramp $Nup 1) simpleGrading (1 30 1)
    //block 7
    hex (10 11 15 14 26 27 31 30) ($Nuu $Nup 1) simpleGrading 
    (
        (
            (0.20       0.6    4)
            (0.80       0.4    1)
        )
        30       
        1
    )
);

edges
("""

# End
diff_bMD3 = """
);

boundary
(
    inlet
    {
        type patch;
        faces
        (
            (12 28 24 8)
            (8 24 20 4)
            (4 20 16 0)
        );
    }   

    outlet
    {
        type patch;
        faces
        (
            (31 15 11 27)
        );
    }   

    compressor
    {
        type patch;
        faces
        (
            (23 7 3 19)
        );
    }   

    upper
    {
        type patch;
        faces
        (
            (28 12 13 29)
            (29 13 14 30)
            (30 14 15 31)
        );
    }   

    lower
    {
        type patch;
        faces
        (
            (0 16 17 1)
        );
    }   

    cowl
    {
        type wall;
        faces
        (
            (10 26 27 11)
            (26 10 6 22)
            (22 6 7 23)
        );
    }   

    axis
    {
        type wall;
        faces
        (
            (18 2 1 17)
            (19 3 2 18)
        );
    }   
);

// ************************************************************************* //
"""

## PreProcess(RunOFv4)

In [9]:
import sys
import os
import numpy as np
from scipy.interpolate import interp1d

class PreProcess(RunOFv4):
    def __init__(self, x, gen, mesher, nProc):
        self.x = x
        self.gen = gen
        self.mesher = mesher
        self.nProc = nProc
        
        if nProc>1:   
            with open('./cases/base-case/system/decomposeParDict', 'w') as f:
                f.write(OF_decomposeParDict(nProc))
                
        try: 
            os.mkdir('./cases/gen%i/data'%self.gen) 
        except OSError as error: 
            pass
            #print(error)

        #super().__init__()
        #self.gen = RunOFv4.gen
    
    def main(self, case):
        if case == 'cylinder':
            self.cylinder()
        elif case == 'diffuser':
            self.diffuser()
        elif case == 'airfoil':
            self.airfoil()
        else:
            print('Pre-Process: case not defined')
        
    ################################################################################
    ######################## MAIN FUNCTIONS ########################################
    ################## Customized for each case ####################################
    ################################################################################
    def cylinder(self):
        # run mesh for base-case 
        if self.gen is 0: 
            self.meshCase('./cases/base-case')
        self.genCaseFiles()
        self.createAFC()

    def diffuser(self):
        def main():
            # run mesh for base-case 
            if self.gen is 0: 
                self.meshCase('./cases/base-case')
            # copy base case for each individual 
            self.genCaseFiles()
            blockMeshGenerator()
        
        def blockMeshGenerator(self):
            ######################## MAIN BODY #########################################
            for ind in range(len(self.x)):
                # Delete the previous blockMeshDict
                os.system("rm ./cases/gen%i/ind%i/system/blockMeshDict" %(self.gen, ind))

                L = self.x[ind,0]
                theta = self.x[ind,1]
                # Interpolate the inner region of the diffuser
                triX = np.array([2+L*np.cos(np.deg2rad(theta)),
                                 2.5+L*np.cos(np.deg2rad(theta)),
                                 4+L*np.cos(np.deg2rad(theta))])
                triY = np.array([L*np.sin(np.deg2rad(theta)),
                                 (4*L*np.sin(np.deg2rad(theta))+0.1)/5,
                                 0.1])
                f2 = interp1d(triX, triY, kind='quadratic')
                x = np.linspace(2+L*np.cos(np.deg2rad(theta)),
                                4+L*np.cos(np.deg2rad(theta)),100)
                y = f2(x)


                # Writing the data in the file
                with open('./cases/gen%i/ind%i/system/blockMeshDict' %(self.gen, ind), "a") as bMD:
                    bMD.write(diff_bMD1)

                    bMD.write('\nLsO %.8f;\nULcO %.8f;\nDLcO %.8f;\nTLcO %.8f;\n' 
                              %(L*np.sin(np.deg2rad(theta)),1.95+L*np.cos(np.deg2rad(theta)),
                                2+L*np.cos(np.deg2rad(theta)),4+L*np.cos(np.deg2rad(theta))))

                    bMD.write(diff_bMD2)

                    bMD.write('    spline 2 3 ( \n')
                    for i in range(len(x)):
                        bMD.write('        (%.8f %.8f 0.0) \n' %(x[i], y[i]))
                    bMD.write('        ) \n')
                    bMD.write('    spline 18 19 ( \n')
                    for i in range(len(x)):
                        bMD.write('        (%.8f %.8f 0.1) \n' %(x[i], y[i]))
                    bMD.write('        ) \n')    

                    bMD.write(diff_bMD3)

                # blockMesh and paraFoam calling
                os.system('cd ./cases/gen%i/ind%i \n blockMesh' %(self.gen, ind))
                #os.system("blockMesh -case gen%i/ind%i/ >./gen%i/ind%i/BMg%ii%i 2>&1" %(gen, ind, gen, ind, gen, ind))
        main()


    #####################################################
    #                    AIRFOIL                        #
    #####################################################
    def airfoil(self):
        # run mesh for base-case 
        if self.gen is 0: 
            self.meshCase('./cases/base-case')
        self.genCaseFiles()
        
        def main():    
            for ind in range(len(self.x)):
                x_cent = self.x[ind,0]
                y_cent = self.x[ind,1]

                # Delete previous blockMeshDict
                os.system("rm ./cases/gen%i/ind%i/system/blockMeshDict" %(self.gen, ind))

                ################################################################################
                #                            PARAMETER DECLARATION                             # 
                ################################################################################
                # Reference percentage of the chord
                pc = 0.25;
                # Inlet circular radius zone
                r = 6;
                # Mesh length
                l = 10;
                # Maximum z-direction coordinate
                maxZ = 0.1;
                # Minimum z-direction coordinate
                minZ = 0.0;

                # Number of cell in the 0-pc of the chord
                Npre = 100;
                # Number of cell in the pc-1 of the chord
                Npost = 70;
                # Number of cell in the wake 
                Nwake = 165;
                # Number of cells in the vertical direction (Ny up and Ny down)
                Ny = 80;
                # Number of cells in the spanwise direction
                Nz = 1;

                # Expansion ratio in the first half of [0,pc] with 65% of the cells
                pre1 = 1;
                # Expansion ratio in the last half of [0,pc] with 35% of the cells
                pre2 = 3.2;
                # Expansion ratio in the last half of [pc,1] with 55% of the cells
                post1 = 1.8;
                # Expansion ratio in the last half of [pc,1] with 45% of the cells
                post2 = 0.5;
                # Expansion ratio in the wake
                wake = 30;
                # Expansion ratio close to the airfoil: 0.2 of [0,l] with 50% of the cells
                yDir1 = 30;
                # Expansion ratio close to the airfoil: 0.8 of [0,l] with 50% of the cells
                yDir2 = 1;
                # Expansion ratio in the spanwise direction
                zDir = 1;

                # Airfoil coordinate set of points
                xTemp, yTemp = joukowsky(x_cent, y_cent)
                x, y = airfoil_correction(xTemp, yTemp)

                # Location of the two pc-intercepts for the airfoil
                firstPC = np.argwhere(x-pc<0)[0]
                lastPC = np.argwhere(x-pc<0)[-1]

                # Get the position of the y-intercept point
                if np.any(y == 0):
                    zeroLoc = np.argwhere(y==0)[1]
                else:
                    zeroLoc = np.argwhere(y<0)[0]

                # Depending on the sign of y at pc, get the y-components for the x_pc
                if y[lastPC] < 0:
                    ypcPos = y[firstPC]
                    ypcNeg = y[lastPC]
                else:
                    ypcPos = y[lastPC]
                    ypcNeg = y[firstPC]

                # Avoid excessive refinement in the leading edge
                y = np.delete(y, np.argwhere(x < 0.0005))
                x = np.delete(x, np.argwhere(x < 0.0005))
                points = np.vstack((x,y)).T

                # Separate between the upper and lower airfoil surfaces
                upper = points[:np.argwhere(y < 0)[0][0], :]
                lower = points[np.argwhere(y < 0)[0][0]:, :]

                # Separate between the 4 quadrants specified with the y = 0 line and the specified pc
                UL = upper[upper[:,0]<pc,:]
                UR = upper[upper[:,0]>pc,:]
                LL = lower[lower[:,0]<pc,:]
                LR = lower[lower[:,0]>pc,:]

                # Sort the 4 quadrants with increasing x-component
                UL = UL[np.argsort(UL[:,0]),:]
                UR = UR[np.argsort(UR[:,0]),:]
                LL = LL[np.argsort(LL[:,0]),:]
                LR = LR[np.argsort(LR[:,0]),:]

                # Fix the first and last points of every quadrant
                UL[0,:] = np.array([0,0])
                UL[-1,:] = np.array([pc,ypcPos])
                UR[0,:] = np.array([pc,ypcPos])
                UR[-1,:] = np.array([1,0])
                LL[0,:] = np.array([0,0])
                LL[-1,:] = np.array([pc,ypcNeg])
                LR[0,:] = np.array([pc,ypcNeg])
                LR[-1,:] = np.array([1,0])

                ################################################################################
                #                                   MAIN BODY                                  # 
                ################################################################################

                # Writing the data in the file
                with open('./cases/gen%i/ind%i/system/blockMeshDict' %(self.gen, ind), "a") as bMD:
                    # Header
                    bMD.write(af_bMD0)

                    # Parameters
                    bMD.write("pc 			%.8f; \n" %pc)
                    bMD.write("rPos		%.8f; \n" %r)
                    bMD.write("rNeg 		%.8f; \n" %(-r))
                    bMD.write("l			%.8f; \n" %l)
                    bMD.write("ypcPos		%.8f; \n" %ypcPos)
                    bMD.write("ypcNeg		%.8f; \n" %ypcNeg)
                    bMD.write("maxZ		%.8f; \n" %maxZ)
                    bMD.write("minZ		%.8f; \n" %minZ)
                    bMD.write("circPos		%.8f; \n" %(r*np.cos(np.pi/4)))
                    bMD.write("circNeg		%.8f; \n" %(-r*np.cos(np.pi/4)))
                    bMD.write("Npre		%i; \n" %Npre)
                    bMD.write("Npost		%i; \n" %Npost)
                    bMD.write("Nwake		%i; \n" %Nwake)
                    bMD.write("Ny		     %i; \n" %Ny)
                    bMD.write("Nz 			%i; \n" %Nz)
                    bMD.write("pre1		%.8f; \n" %pre1)
                    bMD.write("pre2		%.8f; \n" %pre2)
                    bMD.write("post1		%.8f; \n" %post1)
                    bMD.write("post2		%.8f; \n" %post2)
                    bMD.write("wake		%.8f; \n" %wake)
                    bMD.write("yDir1		%.8f; \n" %yDir1)
                    bMD.write("yDir2		%.8f; \n" %yDir2)
                    bMD.write("zDir		%.8f; \n" %zDir)

                    # Vertices
                    bMD.write(af_bMD1)

                    # Blocks
                    bMD.write(af_bMD2)

                    # Edges
                    bMD.write("""
                    edges 
                (
                    // Upper circular inlet
                    arc 4  9  ($circNeg $circPos $minZ)
                    arc 16 21 ($circNeg $circPos $maxZ)

                    // Lower circular inlet
                    arc 4  0  ($circNeg $circNeg $minZ)
                    arc 16 12 ($circNeg $circNeg $maxZ)

                """);

                    # Upper right airfoil
                    bMD.write('    // Upper right airfoil\n')  
                    bMD.write('    spline 8 6 ( \n')
                    for i in range(len(UR)):
                        bMD.write('        (%.8f %.8f %.8f) \n' %(UR[i,0], UR[i,1], minZ))
                    bMD.write('        ) \n')
                    bMD.write('    spline 20 18 ( \n')
                    for i in range(len(UR)):
                        bMD.write('        (%.8f %.8f %.8f) \n' %(UR[i,0], UR[i,1], maxZ))
                    bMD.write('        ) \n')

                    # Upper left airfoil
                    bMD.write('    // Upper left airfoil\n')  
                    bMD.write('    spline 5 8 ( \n')
                    for i in range(len(UL)):
                        bMD.write('        (%.8f %.8f %.8f) \n' %(UL[i,0], UL[i,1], minZ))
                    bMD.write('        ) \n')
                    bMD.write('    spline 17 20 ( \n')
                    for i in range(len(UL)):
                        bMD.write('        (%.8f %.8f %.8f) \n' %(UL[i,0], UL[i,1], maxZ))
                    bMD.write('        ) \n')

                    # Lower left airfoil
                    bMD.write('    // Lower left airfoil\n')
                    bMD.write('    spline 5 3 ( \n')
                    for i in range(len(LL)):
                        bMD.write('        (%.8f %.8f %.8f) \n' %(LL[i,0], LL[i,1], minZ))
                    bMD.write('        ) \n')

                    bMD.write('    spline 17 15 ( \n')
                    for i in range(len(LL)):
                        bMD.write('        (%.8f %.8f %.8f) \n' %(LL[i,0], LL[i,1], maxZ))
                    bMD.write('        ) \n')

                    # Lower right airfoil
                    bMD.write('    // Lower right airfoil\n')
                    bMD.write('    spline 3 6 ( \n')
                    for i in range(len(LR)):
                        bMD.write('        (%.8f %.8f %.8f) \n' %(LR[i,0], LR[i,1], minZ))
                    bMD.write('        ) \n')

                    bMD.write('    spline 15 18 ( \n')
                    for i in range(len(LR)):
                        bMD.write('        (%.8f %.8f %.8f) \n' %(LR[i,0], LR[i,1], maxZ))
                    bMD.write('        ) \n')

                    bMD.write(""");
                    \n""")

                    # Boundary 
                    bMD.write(af_bMD3)

                # blockMesh and paraFoam calling
                os.system("blockMesh -case cases/gen%i/ind%i > BMg%ii%i 2>&1  & wait;" %(self.gen, ind, self.gen, ind))
                os.system("mv BMg%ii%i cases/gen%i/ind%i/BMg%ii%i" %(self.gen, ind, self.gen, ind, self.gen, ind))

        ################################################################################
        #                              Airfoil Functions                               # 
        ################################################################################
        def joukowsky(x_cent,y_cent):
            '''Joukowsky airfoil components calculation (fixed radius)

            INPUTS:
            x_cent: float with the x-position of the center
            y_cent: float with the y-position of the center

            OUTPUT:
            x1:     cartesian coordinates for horizontal axis
            y1:     cartesian coordinates for vertical axis

            This function computes the cartesian coordinates of a Joukowsky 
            airfoil from the position of the center, having always the radius
            fixed so it can go through the points |0,1|
            '''

            # Circle parameters definition
            center = np.array([x_cent,y_cent])
            radius1 = np.sqrt((center[0]-1)**2+(center[1]-0)**2)
            # Second circle will be neglected

            # Circle coordinates calculations
            angle = np.linspace(0,2*np.pi,500)
            chi1 = center[0] + radius1*np.cos(angle)
            eta1 = center[1] + radius1*np.sin(angle)

            # Cartesian components of the Joukowsky transform
            x1 = ((chi1)*(chi1**2+eta1**2+1))/(chi1**2+eta1**2)
            y1 = ((eta1)*(chi1**2+eta1**2-1))/(chi1**2+eta1**2)

            return x1, y1

        def airfoil_correction(x, y):
            '''Set the airfoil chord to 1 and the leading edge to (0,0)

            INPUT:
            x:     cartesian coordinates for horizontal axis
            y:     cartesian coordinates for vertical axis

            OUTPUT:
            xCorr: corrected cartesian coordinates for horizontal axis
            yCorr: corrected cartesian coordinates for vertical axis

            The function scales the airfoil to match the chord dimension to
            a value of 1 (proportionally scaling the vertical dimension). The
            leading edge of the airfoil is moved after to a position in (0,0)
            '''
            # Compute the scale factor (actual chord length)
            c = np.max(x)-np.min(x)

            # Leading edge current position
            LE = np.min(x/c)

            # Corrected position of the coordinates
            xCorr = x/c-LE
            yCorr = y/c

            return xCorr, yCorr
            
        main()

    ################################################################################
    #                        Pre-Process Functions                                 #
    ################################################################################
    def meshCase(self, path):
        os.system('cd '+path+' \n'+self.mesher)
        
    # copy base-case to create case for each individual in population
    def genCaseFiles(self):
        
        for ind in range(len(self.x)):
            #os.system('foamCloneCase cases/base-case cases/gen%i/ind%i' %(self.gen,ind))
            os.system('mkdir -p cases/gen%i/ind%i' %(self.gen, ind))
            os.system('cp -r cases/base-case/0 cases/gen%i/ind%i'  %(self.gen, ind))
            os.system('cp -r cases/base-case/constant cases/gen%i/ind%i'  %(self.gen, ind))
            os.system('cp -r cases/base-case/system cases/gen%i/ind%i'  %(self.gen, ind))  
    
    def changeBC(self):
        # change velocity of moving wall 

        f=ParsedParameterFile('cases/gen'+self.gen+'/ind'+ind+'/0/U')
 
        for b in f["boundaryField"]:
            if "movingWall" in b:
                f["boundaryField"][b]["value"]='uniform ('+wallVel+' 0 0)'
                f["boundaryField"][b]["type"]="fixedValue"
 
        f.writeFile()
        # alternative option: OpenFOAM Python Parser (Ofpp)
    
    def createAFC(self):
        for ind in range(len(self.x)):
            amp = self.x[ind,0]
            freq = self.x[ind,1]#[ind][1]
            print('ind-%i: amp-%.4f freq-%.4f' %(ind, amp, freq))

            #endTime = 150 #sec
            #deltaT = 0.005 #sec
            #timeSteps = int(endTime/deltaT)        
            timeSteps = 1000 #????????????????????????

            time = np.linspace(0,110,timeSteps)
            vel = amp*np.sin(freq*time)

            # save new parameters to case file
            #os.mkdir('./cases/gen%i/ind%i/0')
            with open('./cases/gen%i/ind%i/0/AFCvalues' %(self.gen, ind), 'w') as file:
                file.write('(\n')
                # Initial velocity of the AFC
                file.write('    (0.0 (0.0 0.0 0.0))\n')
                for i in range(timeSteps):
                    file.write('    (%3.5f (%2.5f 0.0 0.0))\n' %(time[i]+40, vel[i]))
                file.write(')')      

# Execute Simulations

In [10]:
class ExecuteSimulations(RunOFv4):
    
    def __init__(self, x, gen, solver, procLim, nProc):
        self.x = x
        self.gen = gen
        self.solver = solver
        self.nProc = nProc
        self.procLim = procLim
    
    def run(self):
        if self.nProc > 1:
            self.parallelRun()
        else:
            self.serialRun()
    
    def wait(self, pids):
        #print('WAITING')
        for pid in pids:
            pid.wait()
    
    ###### Run individuals in parallel ######
    def parallelRun(self):
        ###################### FUNCTIONS ##################################
        ##### Decompose mesh #####
        def decompMesh():
            # If parallel computing is desired, the mesh must be first decomposed
            for ind in range(len(self.x)):
                #print('.## decomposePar for individual %i ...' % ind)
                #os.system('cd cases/gen%i/ind%i \n decomposePar', %(self.gen, self.ind))
                os.system('decomposePar -case ind%i' %ind)
                #os.system('decomposePar -case ind%i > DPg%ii%i 2>&1' %(ind, gen, ind))
                #os.system('mv DPg%ii%i ind%i/DPg%ii%i' %(gen, ind, gen, ind))
            
        # Recompose mesh for each individual
        def recompMesh():
            # If parallel computing is desired, the mesh must be reconstructed after 
            for ind in range(len(self.x)):
                #print('.## reconstructPar for individual %i ...' %ind)
                os.system('reconstructPar -case ind%i' %(ind, self.gen, ind))
                #os.system('reconstructPar -case ind%i > RPg%ii%i 2>&1 &' %(ind, self.gen, ind))
                #mv RPg"$gen"i"$ind" ind$ind/RPg"$gen"i"$ind"
        ###########################################################################################
        ########################## MAIN BODY #################################################
        #print('PARALLEL RUN')
        # Move to the generation folder
        ogDir = os.getcwd() # original directory
        os.chdir('./cases/gen%i' %self.gen) # os.mkdir()
        
        # If parallel computing is desired, the mesh must be first decomposed
        decompMesh()
        
        # All processors will be queued until all are used
        ind = 0
        currentP = 0
        pids = []
        
        while ind < len(self.x):
            if currentP != self.procLim: #currentP < procLim:
                # Send another simulation
                pid = subprocess.Popen(['mpirun','-np',str(self.nProc),self.solver,'-case','ind%i'%ind,'-parallel'])
                #os.system('mpirun -np %i %s -case ind%i -parallel > RUNg%ii%i 2>&1 &'
                #        %(self.nProc, self.solver, ind, self.gen, ind))
                # Store the PID of the above process
                pids.append(pid)
                #print('## Sending ind%i to simulation...' %ind)
                # counters
                ind+=1
                currentP = currentP + self.nProc
            # Then, wait until completion and fill processors again
            else:
                # Wait until all PID in the list has been completed
                self.wait(pids)
                # Delete the current array with all PID
                pids.clear()
                # Restart the number of processors currently in use
                currentP=0
                
        # Wait until all PID in the list has been completed
        self.wait(pids)
        ##### Recompose mesh ##### 
        recompMesh()
        # Return to main directory        
        os.chdir(ogDir)
    
    # run individual's case 
    def serialRun(self):
        #print('SERIAL RUN')
        ##### Serial individual computing #####
        # Move to the generation folder
        ogDir = os.getcwd()
        os.chdir('./cases/gen%i' % self.gen)
        
        # All processors will be queued until all are used 
        pids = []
        ind = 0
        currentP=0
        while ind < len(self.x):
            # If all processors are not in use yet
            if currentP != self.procLim: #currentP < procLim:
                # Send another simulation
                pid = subprocess.Popen([self.solver,'-case','ind%i'%ind])
                #os.system('%s -case ind%i > RUNg%ii%i 2>&1 &' %(self.solver, ind, self.gen, ind))
                # Store the PID of the above process
                pids.append(pid)
                # counters
                currentP = currentP + self.nProc
                ind+=1
            # Then, wait until completion and fill processors again
            else:
                # Wait until all PID in the list has been completed
                self.wait(pids)
                # Delete the current array with all PID
                pids.clear()
                # Restart the number of processors currently in use
                currentP = 0
        
        # Wait until all PID in the list has been completed
        self.wait(pids)
        
        # Return to main directory
        os.chdir(ogDir)

# Post-Process

In [11]:
#from paraview.simple import *
import matplotlib
#This is required to 'plot' inside the CLI
matplotlib.use('AGG')

import numpy as np
import matplotlib.pyplot as plt
from parse import *
import os
import sys


class PostProcess(RunOFv4):
    def __init__(self, x, gen):
        self.x = x
        self.gen = gen
        self.obj = []
        #self.case
    
    def main(self, case):
        for ind in range(len(self.x)):
            if case == 'cylinder':
                obj_i = self.cylinder(ind)
            elif case == 'diffuser':
                obj_i = self.diffuser(ind)
            elif case == 'airfoil':
                obj_i = self.airfoil(ind)
            else:
                print('Post-Process: case not found!')
            
            self.saveText(ind, obj_i)
            self.obj.append(obj_i)
            
        return self.obj
    
    ################################################################################
    ######################## MAIN FUNCTIONS ########################################
    ################## Customized for each case ####################################
    ################################################################################
    

    
    ################################################################################
    #                   Post-Process Universal Functions                           #
    ################################################################################
    
    #### Potential Functions in Development ####
    
    #def readFile(self, path):
        #return data # numpy array
    #def getTimePeriod(self, timeRange):
        #return data # numpy array
    #def applyFunction(fun, data):
        #return obj1, obj2
    #def saveData(self, ind, obj_i):
            #def saveFig():
                # Save figure 
#                 fig, (ax1) = plt.subplots(1, figsize=(10,8))

#                 ax1.plot(forces[timestp40+fx:,0],forces[timestp40+fx:,1]+forces[timestp40+fx:,4],'b',linewidth=2,label='Pressure Force X')             # ax1.plot(forces[timestp40+fx:,0],np.mean(forces[timestp40+fx:,1])*np.ones(len(forces[timestp40+fx:,0])),':b',linewidth=1)
#                 ax1.plot(forces[timestp40+fy:,0],forces[timestp40+fy:,2]+forces[timestp40+fy:,5],'g',linewidth=2,label='Pressure Force Y')
#                 # ax1.plot(forces[timestp40+fy:,0],np.mean(forces[timestp40+fy:,2])*np.ones(len(forces[timestp40+fy:,0])),':g',linewidth=1)
#                 ax1.set_xlabel('Time (s)', fontsize=16)
#                 ax1.set_ylabel(r'Force ($N$)', fontsize=16)
#                 ax1.legend(loc='lower left', fontsize=16)
#                 ax1.tick_params(labelsize=14)
#                 ax1.set_ylim([-0.3,0.3])
#                 ax1.set_xlim([0,150])
#                 # Save figure 
#                 fig, (ax1) = plt.subplots(1, figsize=(10,8))
#                 plt.savefig('./cases/gen%i/data/OSCg%ii%i.png' %(self.gen, self.gen, ind), bbox_inches='tight', dpi=100)
    def saveText(self, ind, obj_i):
#         try: 
#             os.mkdir('./cases/gen%i/data'%self.gen) 
#         except OSError as error: 
#             print(error)           
        # Save fitness to text file 
        np.savetxt('./cases/gen%i/data/FITg%ii%i.txt' %(self.gen, self.gen, ind),
                   np.array(obj_i))
    def paraview(self):
        # OpenFOAM file generation
        os.system('touch cases/gen%i/ind%i/g%ii%i.OpenFOAM'
                 %(self.gen, ind, self.gen, ind))
        ### paraview batch mode ###
        L = self.x[ind,0]
        theta = self.x[ind,1]
        L_diff = 4+L*np.cos(np.deg2rad(theta))
        print('L_diff: %f' %L_diff)
        os.system('pvbatch paraviewPostProcess.py %i %i %.8f' 
                  %(self.gen, ind, L_diff))

    

### Post-Process: Airfoil Case Functions

### Post-Process: Diffuser Case Functions

In [12]:
def airfoil(self, ind):
    # Package importation
    from scipy import integrate  

    # MAIN FUNCTION
    def main():
        #print('AIRFOIL')
        lift, drag = forcePlotAnalysis(ind)
        #area = areaCalc(ind)

        obj_i = [-lift, drag]
        return obj_i
    

    ######################################################################
    ################### Airfoil Case Function ############################
    def forcePlotAnalysis(ind):
        def main():
            #print('FORCE PLOT ANALYSIS')
            ################################################################################
            #                                    INPUTS                                    # 
            ################################################################################
            # Define the expected header of the forces.dat file
            header = ["time",
                      "pressureF_x","pressureF_y","pressureF_z",
                      "viscousF_x","viscousF_y","viscousF_z",
                      "porousF_x","porousF_y","porousF_z",
                      "pressureM_x","pressureM_y","pressureM_z",
                      "viscousM_x","viscousM_y","viscousM_z",
                      "porousM_x","porousM_y","porousM_z"]

            # Give a example line to parse
            exampleLine = "{}\t(({} {} {}) ({} {} {}) ({} {} {})) (({} {} {}) ({} {} {}) ({} {} {}))\n"

            # Read the file and store it in "lines"
            lines = []
            with open ('./cases/gen%i/ind%i/postProcessing/forces/0/forces.dat' %(self.gen, ind), 'rt') as in_file:  
                for line in in_file: 
                    lines.append(line) #Appends current line into lines
            # forces matrix will have len(lines) rows and 19 columns:
            # time-Fxp-Fyp-Fzp-Fxv-Fyv-Fzv-Fxp-Fyp-Fzp-Mxp-Myp-Mzp-Mxv-Myv-Mzv-Mxp-Myp-Mzp 
            forces = np.zeros((len(lines), 19))

            # Iterate along the number of rows (timesteps)
            for i in range(len(lines)):
                # Check if there is ocurrences in the line
                if parse(exampleLine, lines[i]) != None:
                    # If there are ocurrences, copy them into the array
                    for j in range(19):
                        forces[i,j] = float(parse(exampleLine, lines[i])[j])

            # Remove the zeros in the matrix that may exist due to preallocation
            forces = forces[forces[:,0] != 0]
            
            D = forces[-1,1]+forces[-1,4]
            L = forces[-1,2]+forces[-1,5]
            
            # Boundary conditions
            U = 30 # m/s
            rho = 1.225 #kg/m^3
            # area of airfoil
            A = areaCalc(ind) # m^3
            CL = (L*2)/(rho*U**2*A) # [N]/([kg/m^3][m/s]^2[m^2])=unitless
            CD = (D*2)/(rho*U**2*A)
            
#             print('D: ')
#             print(D)
#             print('L: ')
#             print(L)
#             print('lift:')
#             print(lift)
#             print('drag:')
#             print(drag)            
            ###### PLOTTING OPTION ######
            #plotting(forces)

            return L, D
        
            

            ################################################################################
            #                                FITNESS DATA                                  # 
            ################################################################################
#             np.savetxt('./cases/gen%i/data/FITg%ii%i.txt' %(self.gen, self.gen, ind),
#                       np.array((D,L)))

        def plotting(forces):
            ################################################################################
            #                                   PLOTTING                                   # 
            ################################################################################
            # Fancy plot settings
#             plt.style.use('seaborn-deep')
#             plt.style.use('classic')
#             matplotlib.rcParams['axes.linewidth'] = 1.3
#             matplotlib.rcParams['lines.linewidth'] = 1.3
#             matplotlib.rc('text', usetex=True)
#             matplotlib.rcParams['text.latex.preamble'] = [r"\usepackage{amsmath}"]
#             matplotlib.rcParams.update({'font.size': 8})

            # Figure definition
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(20,15))
            fig.subplots_adjust(wspace=0.3, hspace=0.25)

            # Upper left subplot
            ax1.plot(forces[:,0],forces[:,1],'b',linewidth=3,label='Pressure Force X')
            ax1.plot(forces[:,0],forces[:,2],'g',linewidth=3,label='Pressure Force Y')
            ax1.set_ylim(-1*np.max(np.abs([forces[-1,1:3]])),2*np.max(np.abs([forces[-1,1:3]])))
            ax1.set_xlabel(r'Iteration Number', fontsize=24)
            ax1.set_ylabel(r'Force (N)', fontsize=24)
            ax1.legend(loc='upper right', fontsize=20)
            ax1.tick_params(labelsize=26)

            # Upper right subplot
            ax2.plot(forces[:,0],forces[:,4],'r',linewidth=3,label='Viscous Force X')
            ax2.set_ylim(0,2*np.abs([forces[-1,4]]))
            ax2.set_xlabel(r'Iteration Number', fontsize=24)
            ax2.set_ylabel(r'Force (N)', fontsize=24)
            ax2.legend(loc='upper right', fontsize=20)
            ax2.tick_params(labelsize=26)

            # Lower left subplot
            ax3.plot(forces[:,0],forces[:,12],'m',linewidth=3,label='Pressure Moment Z')
            ax3.set_ylim(-2*np.abs([forces[-1,12]]),2*np.abs([forces[-1,12]]))
            ax3.set_xlabel(r'Iteration Number', fontsize=24)
            ax3.set_ylabel(r'Moment (N m)', fontsize=24)
            ax3.legend(loc='lower right', fontsize=20)
            ax3.tick_params(labelsize=26)

            # Lower right subplot
            ax4.plot(forces[:,0],forces[:,15],'k',linewidth=3,label='Viscous Moment Z')
            ax4.set_ylim(-2*np.abs([forces[-1,15]]),2*np.abs([forces[-1,15]]))
            ax4.set_xlabel(r'Iteration Number', fontsize=24)
            ax4.set_ylabel(r'Moment (N$\cdot$m)', fontsize=24)
            ax4.legend(loc='upper right', fontsize=20)
            ax4.tick_params(labelsize=26)

            # General figure title
            fig.suptitle('Generation %i Individual %i' %(self.gen, ind), fontsize=30)

            # Figure saving
            plt.savefig('./cases/gen%i/data/CONg%ii%i.png' %(self.gen, self.gen, ind), bbox_inches='tight', dpi=100)

        lift, drag = main()
        return lift, drag

    def areaCalc(ind):
        def main():
            ################################################################################
            #                                    INPUTS                                    # 
            ################################################################################
            # Get the inputs from the terminal line
            x_cent = self.x[ind,0]
            y_cent = self.x[ind,1]

            ################################################################################
            #                            PARAMETER DECLARATION                             # 
            ################################################################################
            # Reference percentage of the chord
            pc = 0.25;

            # Airfoil coordinate set of points
            xTemp, yTemp = joukowsky(x_cent, y_cent)
            x, y = airfoil_correction(xTemp, yTemp)

            # Location of the two pc-intercepts for the airfoil
            firstPC = np.argwhere(x-pc<0)[0]
            lastPC = np.argwhere(x-pc<0)[-1]

            # Get the position of the y-intercept point
            if np.any(y == 0):
                zeroLoc = np.argwhere(y==0)[1]
            else:
                zeroLoc = np.argwhere(y<0)[0]

            # Depending on the sign of y at pc, get the y-components for the x_pc
            if y[lastPC] < 0:
                ypcPos = y[firstPC]
                ypcNeg = y[lastPC]
            else:
                ypcPos = y[lastPC]
                ypcNeg = y[firstPC]

            # Avoid excessive refinement in the leading edge
            y = np.delete(y, np.argwhere(x < 0.0005))
            x = np.delete(x, np.argwhere(x < 0.0005))
            points = np.vstack((x,y)).T

            # Separate between the upper and lower airfoil surfaces
            upper = points[:np.argwhere(y < 0)[0][0], :]
            lower = points[np.argwhere(y < 0)[0][0]:, :]

            # Separate between the 4 quadrants specified with the y = 0 line and the specified pc
            UL = upper[upper[:,0]<pc,:]
            UR = upper[upper[:,0]>pc,:]
            LL = lower[lower[:,0]<pc,:]
            LR = lower[lower[:,0]>pc,:]

            # Sort the 4 quadrants with increasing x-component
            UL = UL[np.argsort(UL[:,0]),:]
            UR = UR[np.argsort(UR[:,0]),:]
            LL = LL[np.argsort(LL[:,0]),:]
            LR = LR[np.argsort(LR[:,0]),:]

            # Fix the first and last points of every quadrant
            UL[0,:] = np.array([0,0])
            UL[-1,:] = np.array([pc,ypcPos])
            UR[0,:] = np.array([pc,ypcPos])
            UR[-1,:] = np.array([1,0])
            LL[0,:] = np.array([0,0])
            LL[-1,:] = np.array([pc,ypcNeg])
            LR[0,:] = np.array([pc,ypcNeg])
            LR[-1,:] = np.array([1,0])

            ################################################################################
            #                               AREA CALCULATION                               # 
            ################################################################################
            # Once the different lines are computed, the area will be computed as the integral of those lines

            # In case the lower surface of the airfoil interceps the y = 0 axis, it must be divided so all areas 
            # are computed independently
            lowerNeg = lower[lower[:,1]<0,:]
            lowerPos = lower[lower[:,1]>0,:]

            # Upper surface area
            A1 = integrate.simps(upper[np.argsort(upper[:,0]),1], upper[np.argsort(upper[:,0]),0])
            # Lower surface area for points with negative y
            A2 = -integrate.simps(lowerNeg[np.argsort(lowerNeg[:,0]),1], lowerNeg[np.argsort(lowerNeg[:,0]),0])
            # Possible lower surface area for points with positive y
            A3 = integrate.simps(lowerPos[np.argsort(lowerPos[:,0]),1], lowerPos[np.argsort(lowerPos[:,0]),0])

            # The area will be the sum of the areas and substracting the possible intercept of both
            area = A1 + A2 - A3 

            # Append the area into the FIT file for the generation and individual
            with open('./cases/gen%i/data/FITg%ii%i.txt' %(self.gen, self.gen, ind), "a") as file:
                file.write(str(area))

            return area

        ################################################################################
        #                              FUNCTION DEFINITION                             # 
        ################################################################################
        def joukowsky(x_cent,y_cent):
            '''Joukowsky airfoil components calculation (fixed radius)

            INPUTS:
            x_cent: float with the x-position of the center
            y_cent: float with the y-position of the center

            OUTPUT:
            x1:     cartesian coordinates for horizontal axis
            y1:     cartesian coordinates for vertical axis

            This function computes the cartesian coordinates of a Joukowsky 
            airfoil from the position of the center, having always the radius
            fixed so it can go through the points |0,1|
            '''

            # Circle parameters definition
            center = np.array([x_cent,y_cent])
            radius1 = np.sqrt((center[0]-1)**2+(center[1]-0)**2)
            # Second circle will be neglected

            # Circle coordinates calculations
            angle = np.linspace(0,2*np.pi,500)
            chi1 = center[0] + radius1*np.cos(angle)
            eta1 = center[1] + radius1*np.sin(angle)

            # Cartesian components of the Joukowsky transform
            x1 = ((chi1)*(chi1**2+eta1**2+1))/(chi1**2+eta1**2)
            y1 = ((eta1)*(chi1**2+eta1**2-1))/(chi1**2+eta1**2)

            return x1, y1

        def airfoil_correction(x, y):
            '''Set the airfoil chord to 1 and the leading edge to (0,0)

            INPUT:
            x:     cartesian coordinates for horizontal axis
            y:     cartesian coordinates for vertical axis

            OUTPUT:
            xCorr: corrected cartesian coordinates for horizontal axis
            yCorr: corrected cartesian coordinates for vertical axis

            The function scales the airfoil to match the chord dimension to
            a value of 1 (proportionally scaling the vertical dimension). The
            leading edge of the airfoil is moved after to a position in (0,0)
            '''
            # Compute the scale factor (actual chord length)
            c = np.max(x)-np.min(x)

            # Leading edge current position
            LE = np.min(x/c)

            # Corrected position of the coordinates
            xCorr = x/c-LE
            yCorr = y/c

            return xCorr, yCorr

        area = main()
        return area
    
    obj_i = main()
    return obj_i

PostProcess.airfoil = airfoil

SyntaxError: EOL while scanning string literal (<ipython-input-12-5b10288842b9>, line 20)

In [ ]:
#%%add_to PostProcess
def diffuser(self, ind):
    def main():
        # OpenFOAM file generation
        os.system('touch cases/gen%i/ind%i/g%ii%i.OpenFOAM'
                  %(self.gen, ind, self.gen, ind))
        ### paraview batch mode ###
        L = self.x[ind,0]
        theta = self.x[ind,1]
        L_diff = 4+L*np.cos(np.deg2rad(theta))
        print('L_diff: %f' %L_diff)
        os.system('pvbatch paraviewPostProcess.py %i %i %.8f' 
                  %(self.gen, ind, L_diff))
        ## analysis data file in python ##
        PR, Ma = meanCompValue(ind)
        obj_i = [PR, Ma]
        return obj_i

    def meanCompValue(ind):
        def main():
            # Case parameters
            Uinf = 590 # m/s
            Tinf = 216 # K
            Pinf = 19330 # Pa
            gamma = 1.4

            # Upstream conditions
            Minf = Uinf/np.sqrt(gamma*287*Tinf)
            P0inf = Pinf*(1+(gamma-1)/(2)*Minf**2)**((gamma)/(gamma-1))

            # The expected header of the file is
            header = ['p',  'U_x', 'U_y', 'U_z', 'T', '\alpha_t', '\varepsilon', 'k', '\nu_t', '\rho', 'vtk', 'arc', 'x', 'y', 'z']

            # Read the data file 
            data = np.genfromtxt('./cases/gen%i/ind%i/plotOverLineData.csv' %(self.gen, ind), skip_header=1,delimiter=',')

            #########################################################
            #                   MAIN FUNCTION                       #
            #########################################################
            # Figures preamble
            plt.style.use('seaborn-deep')
            plt.style.use('classic')
            matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler('color', ['#0072B2', '#009E73', '#D55E00', '#CC79A7', '#F0E442', '#56B4E9'])
            matplotlib.rcParams['axes.linewidth'] = 1.3
            matplotlib.rcParams['lines.linewidth'] = 1.3
            matplotlib.rc('text', usetex=True)
            matplotlib.rcParams['text.latex.preamble'] = [r"\usepackage{amsmath}"]
            matplotlib.rcParams.update({'font.size': 8})

            # Analyze the length of the header before going on
            if len(header) == len(np.loadtxt('./cases/gen%i/ind%i/plotOverLineData.csv' 
                                             %(self.gen, ind), 
                                             dtype='str', delimiter=',')[0,:]):  
                # Velocity magnitude computation
                Umag = np.sqrt(data[:,1]**2+data[:,2]**2+data[:,3]**2)
                # Mach computation
                M = Umag/np.sqrt(gamma*287*data[:,4])
                # Velocity magnitude plotting 
                velocityPlot()
                # Total pressure computation
                P0 = data[:,0]*(1+(gamma-1)/(2)*M**2)**((gamma)/(gamma-1))
                # Total pressure plotting 
                pressurePlot()
                # File saving of the values of the simulation
                np.savetxt('./cases/gen%i/data/FITg%ii%i.txt' %(self.gen, self.gen, ind),
                           np.array([np.nanmean(P0)/P0inf,np.nanmean(M)]))

                PR = np.nanmean(P0)/P0inf
                Ma = np.nanmean(M)
                return PR, Ma
            else:
                raise ValueError('Incorrect .csv file!')
        def pressurePlot():
            # Total pressure plotting 
            fig, ax = plt.subplots(1, figsize=(8,6))
            ax.plot(P0/1e3,'r',lw=2,label=r'$P$')
            ax.plot(np.nanmean(P0)/1e3*np.ones(len(P0)),'r:',lw=2,label=r'$\mu_{P}$')
            ax.plot([0,len(data)],[P0inf/1e3, P0inf/1e3], 'k-.',lw=1.5,label=r'$P_{\infty,0}$')
            ax.set_xlabel(r'$y\ (m)$',fontsize=20)
            ax.set_ylabel(r'$P\ (kPa)$',fontsize=20)
            ax.legend(fontsize=16, loc='lower left')
            ax.set_xlim([0,len(data)])
            ax.set_xticks([0,0.5*len(data),len(data)])
            ax.set_ylim([1,180])
            ax.set_xticklabels([r'$0.10$',r'$0.45$',r'$0.80$'])
            ax.tick_params(axis = 'both', labelsize = 18)
            ax.set_title('Total pressure', fontsize = 26)
            plt.savefig('./cases/gen%i/data/Pg%ii%i.png' %(self.gen, self.gen, ind), bbox_inches='tight', dpi=200)
        def velocityPlot():
            # Velocity magnitude plotting 
            fig, ax = plt.subplots(1, figsize=(8,6))
            ax.plot(M,'b',lw=2,label=r'$Ma$')
            ax.plot(np.nanmean(M)*np.ones(len(data[:,2])),'b:',lw=2,label=r'$\mu_{Ma}$')
            ax.set_xlabel(r'$y\ (m)$',fontsize=20)
            ax.set_ylabel(r'$Ma$',fontsize=20)
            ax.legend(fontsize=16, loc='lower left')
            ax.set_xlim([0,len(data)])
            ax.set_xticks([0,0.5*len(data),len(data)])
            ax.set_ylim([0.25,3])
            ax.set_xticklabels([r'$0.10$',r'$0.45$',r'$0.80$'])
            ax.tick_params(axis = 'both', labelsize = 18)
            ax.set_title('Mach number', fontsize = 26)
            plt.savefig('./cases/gen%i/data/Mag%ii%i.png' %(self.gen, self.gen, ind), bbox_inches='tight', dpi=200)

        main()
    obj_i = main()
    return obj_i

PostProcess.diffuser = diffuser

### Post-Process: Cylinder Case Functions

In [ ]:
#%%add_to PostProcess
def cylinder(self, ind):
    def main():
        sigmaX, sigmaY = forcePlotAnalysis(ind)
        obj_i = [sigmaX, sigmaY]
        return obj_i
    def forcePlotAnalysis(self, ind):
        #os.system('postProcess -latestTime')
        def main():
            # Read the file efficientyly
            s = open('./cases/gen%i/ind%i/postProcessing/forces/0/forces.dat' %(self.gen, ind)).read().replace('(',' ').replace(')',' ').replace('\t',' ')
            forces = np.genfromtxt(io.BytesIO(s.encode()))

            # GET            
            timestp40 = int(np.argwhere(forces[:,0]>40)[0])

            matFX = np.invert(forces[timestp40:,1] > forces[-1,1])
            logicFX = np.logical_xor(matFX[0:-2],matFX[1:-1])
            if len(np.argwhere(logicFX))%2 == 1:
                fx = int(np.argwhere(logicFX)[1])
            else:
                fx = int(np.argwhere(logicFX)[0])

            matFY = np.invert(forces[timestp40:,2] > forces[-1,2])
            logicFY = np.logical_xor(matFY[0:-2],matFY[1:-1])
            if np.sum(logicFY) == 0:
                fy = timestp40
            else:
                if len(np.argwhere(logicFY))%2 == 1:
                    fy = int(np.argwhere(logicFY)[1])
                else:
                    fy = int(np.argwhere(logicFY)[0])


            sigmaX = np.std(forces[timestp40+fx:,1]+forces[timestp40+fx:,4])
            sigmaY = np.std(forces[timestp40+fy:,2]+forces[timestp40+fy:,5])

            #displayData(ind, sigmaX, sigmaY)
            #saveData(sigmaX, sigmaY)

            return sigmaX, sigmaY

        def saveData(self, ind, sigmaX, sigmaY):
            def saveFig():
                # Save figure 
                fig, (ax1) = plt.subplots(1, figsize=(10,8))

                ax1.plot(forces[timestp40+fx:,0],forces[timestp40+fx:,1]+forces[timestp40+fx:,4],'b',linewidth=2,label='Pressure Force X')             # ax1.plot(forces[timestp40+fx:,0],np.mean(forces[timestp40+fx:,1])*np.ones(len(forces[timestp40+fx:,0])),':b',linewidth=1)
                ax1.plot(forces[timestp40+fy:,0],forces[timestp40+fy:,2]+forces[timestp40+fy:,5],'g',linewidth=2,label='Pressure Force Y')
                # ax1.plot(forces[timestp40+fy:,0],np.mean(forces[timestp40+fy:,2])*np.ones(len(forces[timestp40+fy:,0])),':g',linewidth=1)
                ax1.set_xlabel('Time (s)', fontsize=16)
                ax1.set_ylabel(r'Force ($N$)', fontsize=16)
                ax1.legend(loc='lower left', fontsize=16)
                ax1.tick_params(labelsize=14)
                ax1.set_ylim([-0.3,0.3])
                ax1.set_xlim([0,150])
                # Save figure 
                fig, (ax1) = plt.subplots(1, figsize=(10,8))
                plt.savefig('./cases/gen%i/data/OSCg%ii%i.png' %(self.gen, self.gen, ind), bbox_inches='tight', dpi=100)
            def saveText():
                # Save fitness to text file 
                np.savetxt('./cases/gen%i/data/FITg%ii%i.txt' %(self.gen, self.gen, ind),
                           np.array([sigmaX, sigmaY]))

            #os.system('mkdir cases/gen%i/data' % self.gen)
            os.mkdir('./cases/gen%i/data' %self.gen)
            saveFig()
            saveText()
        main()
    obj_i = main()
    return obj_i

PostProcess.cylinder = cylinder